## <a href = 'https://8weeksqlchallenge.com/case-study-1/'>Danny's Diner</a>

#### Introduction

<img src = 'https://8weeksqlchallenge.com/images/case-study-designs/1.png'>

<br>

Danny seriously loves Japanese food so in the beginning of 2021, he decides to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen.

Danny’s Diner is in need of your assistance to help the restaurant stay afloat - the restaurant has captured some very basic data from their few months of operation but have no idea how to use their data to help them run the business.

#### Case Study Questions
Each of the following case study questions can be answered using a single SQL statement:

1. What is the total amount each customer spent at the restaurant?
2.  How many days has each customer visited the restaurant?
3. What was the first item from the menu purchased by each customer?
4. What is the most purchased item on the menu and how many times was it purchased by all customers?
5. Which item was the most popular for each customer?
6. Which item was purchased first by the customer after they became a member?
7. Which item was purchased just before the customer became a member?
8. What is the total items and amount spent for each member before they became a member?
9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

#### Startup
Install SQL extension to Jupyter notebook + import libraries.

In [1]:
#!pip install ipython-sql
import sql
from sqlalchemy import create_engine


%load_ext sql
%sql sqlite://

  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=8d4355121eb525c82e9c8bd3f49ec97efcddb02e99b85dd2a83c52ffe46ec5fa
  Stored in directory: c:\users\chloe\appdata\local\pip\cache\wheels\75\f7\28\77a076f1fa8cbeda61aca712815d04d7a32435f04a26a2dd7b
Successfully built prettytable


Tables as per link above

In [2]:
%%sql 

CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');

 * sqlite://
Done.
15 rows affected.


[]

In [3]:
%%sql

CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

 * sqlite://
Done.
2 rows affected.


[]

In [4]:
%%sql

CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');

 * sqlite://
Done.
3 rows affected.


[]

#### Start
1. What is the total amount each customer spent at the restaurant?

In [6]:
%%sql

SELECT
    s.customer_id,
    SUM(price) AS total_sales
FROM sales AS s
JOIN menu AS m
ON s.product_id = m.product_id
GROUP BY customer_id

 * sqlite://
Done.


customer_id,total_sales
A,76
B,74
C,36


2. How many days has each customer visited the restaurant?

In [31]:
%%sql

SELECT 
    s.customer_id,
    COUNT(DISTINCT(order_date)) AS number_days
FROM sales AS s
GROUP BY s.customer_id;

 * sqlite://
Done.


customer_id,number_days
A,4
B,6
C,2


3. What was the first item from the menu purchased by each customer?

Notes on DENSE_RANK() - <a href = 'https://www.geeksforgeeks.org/postgresql-dense_rank-function/'>here</a>.

In [35]:
%%sql

WITH ranked_sales AS
(SELECT
    s.customer_id,
    s.order_date,
    m.product_name,
    DENSE_RANK() OVER (PARTITION BY s.customer_id
                              ORDER BY s.order_date) AS ranking
FROM sales AS s
JOIN menu AS m
    ON s.product_id = m.product_id)
    
SELECT 
    customer_id,
    product_name
FROM ranked_sales
WHERE ranking = 1
GROUP BY customer_id, product_name;

 * sqlite://
Done.


customer_id,product_name
A,curry
A,sushi
B,curry
C,ramen


4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [36]:
%%sql

SELECT
    (COUNT(s.product_id)) AS product,
    product_name
FROM sales AS s
JOIN menu AS m
    ON s.product_id = m.product_id
GROUP BY s.product_id, product_name
ORDER BY product DESC;

 * sqlite://
Done.


product,product_name
8,ramen
4,curry
3,sushi


5. Which item was the most popular for each customer?

In [42]:
%%sql

WITH most_popular AS
(SELECT
     s.customer_id,
     m.product_name,
     COUNT(s.product_id) AS purchased,
     DENSE_RANK() OVER(PARTITION BY s.customer_id
                      ORDER BY COUNT(s.customer_id) DESC) AS ranking
 FROM sales AS s
 JOIN menu AS m
     ON s.product_id = m.product_id
 GROUP BY s.customer_id, m.product_id, product_name
)

SELECT * FROM most_popular;

 * sqlite://
Done.


customer_id,product_name,purchased,ranking
A,ramen,3,1
A,curry,2,2
A,sushi,1,3
B,sushi,2,1
B,curry,2,1
B,ramen,2,1
C,ramen,3,1


6. Which item was purchased first by the customer after they became a member?

In [55]:
%%sql

WITH ranked_sales AS 
(SELECT s.customer_id, m.join_date, s.order_date, s.product_id,
     DENSE_RANK() OVER(PARTITION BY s.customer_id
                         ORDER BY s.order_date) AS ranked #section out by customer_id, then place them in order of date of order
 FROM sales AS s
 JOIN members AS m
     ON s.customer_id = m.customer_id
 WHERE s.order_date >= m.join_date) #order date must be larger than join date

SELECT s.customer_id, s.order_date, m.product_name
FROM rankeD_sales as s
JOIN menu as m
    ON s.product_id = m.product_id
WHERE s.ranked = 1
ORDER BY s.customer_id

 * sqlite://
Done.


customer_id,order_date,product_name
A,2021-01-07,curry
B,2021-01-11,sushi


7.  Which item was purchased just before the customer became a member?

In [66]:
%%sql

WITH ranked_sales AS 
(SELECT s.customer_id, m.join_date, s.order_date, s.product_id,
     DENSE_RANK() OVER(PARTITION BY s.customer_id
                         ORDER BY s.order_date DESC) AS ranked
 FROM sales AS s
 JOIN members AS m
     ON s.customer_id = m.customer_id
 WHERE s.order_date < m.join_date) 

SELECT s.customer_id, s.order_date, m.product_name
FROM ranked_sales AS s
JOIN menu AS m
    ON s.product_id = m.product_id
WHERE s.ranked = 1
ORDER BY s.customer_id

 * sqlite://
Done.


customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-04,sushi


8. What is the total items and amount spent for each member before they became a member?

In [77]:
%%sql

SELECT 
    s.customer_id, 
    SUM(men.price) AS total_spend, 
    COUNT(DISTINCT s.product_id) AS items
FROM sales AS s
JOIN members AS m
    ON s.customer_id = m.customer_id
JOIN menu AS men
    ON s.product_id = men.product_id
WHERE s.order_date < m.join_date
                         
GROUP BY s.customer_id;                         

 * sqlite://
Done.


customer_id,total_spend,items
A,25,2
B,40,2


9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [82]:
%%sql

WITH commontableexpression_points AS
(
SELECT
    menu.product_id, menu.product_name, 
    CASE WHEN menu.product_name = 'sushi' 
         THEN menu.price * 20
         ELSE price * 10
    END 
    AS points
FROM menu
)

SELECT 
    customer_id, 
    SUM(commontableexpression_points.points) AS total_points
FROM commontableexpression_points
JOIN sales
    ON commontableexpression_points.product_id = sales.product_id
GROUP BY sales.customer_id

 * sqlite://
Done.


customer_id,total_points
A,860
B,940
C,360


10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [92]:
%%sql

WITH cte_days AS
(SELECT members.customer_id, members.join_date,sales.order_date,sales.product_id,
julianday(sales.order_date)-julianday(members.join_date) AS days_diff
from sales 
INNER JOIN members on sales.customer_id = members.customer_id)


SELECT SUM(CASE WHEN cte_days.product_id =1 THEN menu.price*20
        WHEN days_diff BETWEEN 0 and 6 and days_diff >=0 THEN menu.price*20
 WHEN days_diff < 0 THEN menu.price * 10
WHEN cte_days.product_id =1 THEN menu.price*20
WHEN cte_days.order_date >= "2021-02-01" THEN menu.price*0

ELSE menu.price*10 END) AS total_points, cte_days.product_id,  cte_days.customer_id, cte_days.order_date, cte_days.join_date, cte_days.days_diff
FROM cte_days
LEFT JOIN MENU on cte_days.product_id = menu.product_id
GROUP BY customer_id

 * sqlite://
Done.


total_points,product_id,customer_id,order_date,join_date,days_diff
1370,1,A,2021-01-01,2021-01-07,-6.0
820,2,B,2021-01-01,2021-01-09,-8.0
